In [1]:
import numpy as np
import pandas as pd
import datetime
import re
import os

import wrangle


In [2]:
df = pd.read_csv('coast_df.csv')

In [3]:
df2 = df.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118295 entries, 0 to 118294
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  118295 non-null  int64  
 1   datetime    118295 non-null  object 
 2   ercot_load  118294 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.7+ MB


In [5]:
df.head()

,Unnamed: 0,datetime,ercot_load
0,0,2009-01-01 01:00:00.003000,7300.317873
1,1,2009-01-01 01:59:59.997000,7174.769899
2,2,2009-01-01 03:00:00,7108.488956
3,3,2009-01-01 04:00:00.003000,7024.863242
4,4,2009-01-01 04:59:59.997000,7049.567942


In [6]:
# 1. Pull out first 16 characters
df.datetime = df.datetime.str[:16]
# 2. Deconstruct into date and time columns
df['date'] = df.datetime.str[:10]
df['time'] = df.datetime.str[-5:]
# 3. Convert date column into datetime format
df['date'] = pd.to_datetime(df.date)

# 4. Find any time where starts with 24, then for that index, add one day to to date column then update time to 00
#For all indices w/ a time that starts with 24, add one day to the date
df.loc[df.time.str.startswith('24'),'date'] = df.date + datetime.timedelta(days=1)
#Replace all '24:' in time with '00:'
df.time = df.time.apply(lambda x: re.sub(r'24:','00:',x))

# 5. Convert date column back to string and Concat date and time column >> datetime
df['new_datetime'] = df.date.astype(str) + ' ' + df.time

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118295 entries, 0 to 118294
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    118295 non-null  int64         
 1   datetime      118295 non-null  object        
 2   ercot_load    118294 non-null  float64       
 3   date          118295 non-null  datetime64[ns]
 4   time          118295 non-null  object        
 5   new_datetime  118295 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 5.4+ MB


In [8]:
df.head()

,Unnamed: 0,datetime,ercot_load,date,time,new_datetime
0,0,2009-01-01 01:00,7300.317873,2009-01-01,01:00,2009-01-01 01:00
1,1,2009-01-01 01:59,7174.769899,2009-01-01,01:59,2009-01-01 01:59
2,2,2009-01-01 03:00,7108.488956,2009-01-01,03:00,2009-01-01 03:00
3,3,2009-01-01 04:00,7024.863242,2009-01-01,04:00,2009-01-01 04:00
4,4,2009-01-01 04:59,7049.567942,2009-01-01,04:59,2009-01-01 04:59


**NOTES:**  now we have the cleaned new_datetime column, but it's a string

In [9]:
# 6. Convert new datetime column to datetime
df['new_datetime'] = pd.to_datetime(df.new_datetime)

# 7. Perform dt.round
df.new_datetime = df.new_datetime.dt.round("H")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118295 entries, 0 to 118294
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    118295 non-null  int64         
 1   datetime      118295 non-null  object        
 2   ercot_load    118294 non-null  float64       
 3   date          118295 non-null  datetime64[ns]
 4   time          118295 non-null  object        
 5   new_datetime  118295 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 5.4+ MB


In [11]:
df.head()

,Unnamed: 0,datetime,ercot_load,date,time,new_datetime
0,0,2009-01-01 01:00,7300.317873,2009-01-01,01:00,2009-01-01 01:00:00
1,1,2009-01-01 01:59,7174.769899,2009-01-01,01:59,2009-01-01 02:00:00
2,2,2009-01-01 03:00,7108.488956,2009-01-01,03:00,2009-01-01 03:00:00
3,3,2009-01-01 04:00,7024.863242,2009-01-01,04:00,2009-01-01 04:00:00
4,4,2009-01-01 04:59,7049.567942,2009-01-01,04:59,2009-01-01 05:00:00


**NOTES:** Now we have that same info as a datetime and rounded to the nearest hour

In [12]:
df.loc[71807:71811] #Here we see 3am skipped, when it should skip 2am

,Unnamed: 0,datetime,ercot_load,date,time,new_datetime
71807,1679,03/11/2017 24:00,9120.385718,2017-03-12,00:00,2017-03-12 00:00:00
71808,1680,03/12/2017 01:00,8579.644600,2017-03-12,01:00,2017-03-12 01:00:00
71809,1681,03/12/2017 02:00,8251.748166,2017-03-12,02:00,2017-03-12 02:00:00
71810,1682,03/12/2017 04:00,8016.291377,2017-03-12,04:00,2017-03-12 04:00:00
71811,1683,03/12/2017 05:00,7894.080492,2017-03-12,05:00,2017-03-12 05:00:00


In [13]:
df.loc[77518:77522] #Here we see 2am duplicated, when 1am should be duplicated

,Unnamed: 0,datetime,ercot_load,date,time,new_datetime
77518,7390,11/04/2017 24:00,11692.944342,2017-11-05,00:00,2017-11-05 00:00:00
77519,7391,11/05/2017 01:00,11086.848799,2017-11-05,01:00,2017-11-05 01:00:00
77520,7392,11/05/2017 02:00,10677.703425,2017-11-05,02:00,2017-11-05 02:00:00
77521,7393,11/05/2017 02:00,10377.230056,2017-11-05,02:00,2017-11-05 02:00:00
77522,7394,11/05/2017 03:00,10256.523453,2017-11-05,03:00,2017-11-05 03:00:00


In [14]:
#indices we know to be affected
mar_adj_ind = [71809, 80545, 89281, 98017, 106921, 115657]
nov_adj_ind = [68784, 59880, 51144, 42408, 33672, 24936, 16200, 7296, 77520, 86256, 94992, 103728, 112632]

In [15]:
# to the March, we are adding 1 hour to the 2am time 
#. b/c ERCOT skips 3am, when it should skip 2am
df.loc[mar_adj_ind,'new_datetime'] = df.loc[mar_adj_ind,'new_datetime'] + datetime.timedelta(hours=1)

#to the November, we are subtracting 1 hour from the first 2am, 
#  as there are supposed to be two 1ams, not two 2ams
df.loc[nov_adj_ind,'new_datetime'] = df.loc[nov_adj_ind,'new_datetime'] - datetime.timedelta(hours=1)

In [16]:
df.loc[71807:71811] #YAY - now 2am is skipped

,Unnamed: 0,datetime,ercot_load,date,time,new_datetime
71807,1679,03/11/2017 24:00,9120.385718,2017-03-12,00:00,2017-03-12 00:00:00
71808,1680,03/12/2017 01:00,8579.644600,2017-03-12,01:00,2017-03-12 01:00:00
71809,1681,03/12/2017 02:00,8251.748166,2017-03-12,02:00,2017-03-12 03:00:00
71810,1682,03/12/2017 04:00,8016.291377,2017-03-12,04:00,2017-03-12 04:00:00
71811,1683,03/12/2017 05:00,7894.080492,2017-03-12,05:00,2017-03-12 05:00:00


In [17]:
df.loc[77518:77522] #Yay! Now 1am should be duplicated

,Unnamed: 0,datetime,ercot_load,date,time,new_datetime
77518,7390,11/04/2017 24:00,11692.944342,2017-11-05,00:00,2017-11-05 00:00:00
77519,7391,11/05/2017 01:00,11086.848799,2017-11-05,01:00,2017-11-05 01:00:00
77520,7392,11/05/2017 02:00,10677.703425,2017-11-05,02:00,2017-11-05 01:00:00
77521,7393,11/05/2017 02:00,10377.230056,2017-11-05,02:00,2017-11-05 02:00:00
77522,7394,11/05/2017 03:00,10256.523453,2017-11-05,03:00,2017-11-05 03:00:00


In [18]:
#Make Time Zone aware
df = df.set_index('new_datetime').tz_localize(tz='US/Central',ambiguous='infer')

## GENERATES ERROR:
#.  AmbiguousTimeError: 2009-11-01 01:00:00

**NOTES:** Each of these created errors earlier: needed to add them to the nov_adj_ind list as they were double 2ams instead of 1ams

df[df.new_datetime.astype(str).str.startswith('2016-11-06')]

# LET's look at that date:
df.loc[7294:7299]

# LET's look at that date:
df.loc[16198:16202]

# LET's look at that date:
df.loc[24934:24938]

# LET's look at that date:
df.loc[33670:33674]

# LET's look at that date:
df.loc[42406:42410]

# LET's look at that date:
df.loc[51142:51146]

# LET's look at that date:
df.loc[59878:59882]

# LET's look at that date:
df.loc[68782:68786]

**NOTES:** Okay, I've now updated nov_adj_ind with all the other aaffected dates